In [1]:
# stanza.download("fr")
# stanza.download("nl")

In [2]:
from collections import Counter
import pandas as pd
import stanza
import re

In [19]:
en_gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/en/ann/1_lem_bert_uncased_ann_predictions.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [20]:
def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

In [21]:
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({1: 461, 2: 1})
Counter({1: 1064, 2: 766, 3: 357, 4: 120, 5: 31, 6: 18, 7: 4, 8: 1})


In [22]:
max(gts, key=len)

'heart failure with preserved left ventricular ejection fraction'

In [23]:
# gts[gts.str.split(' ').len() == 3]

In [24]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

222 240 2139
462
2361


(48.05, 9.4, 15.72)

Stanza

In [25]:
nlp = stanza.Pipeline(lang='en')# processors='tokenize,mwt,pos,lemma'

2021-04-16 16:40:41 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-16 16:40:41 INFO: Use device: cpu
2021-04-16 16:40:41 INFO: Loading: tokenize
2021-04-16 16:40:41 INFO: Loading: pos
2021-04-16 16:40:41 INFO: Loading: lemma
2021-04-16 16:40:41 INFO: Loading: depparse
2021-04-16 16:40:42 INFO: Loading: sentiment
2021-04-16 16:40:42 INFO: Loading: ner
2021-04-16 16:40:43 INFO: Done loading processors!


In [26]:
def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

In [27]:
preds_lem = lemma(preds)
gts_lem = lemma(gts)

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

After lemmatization
432
2149
211 221 1938
(48.84, 9.82, 16.35)


In [28]:
set(preds_lem)

{"'s",
 'absorptiometry',
 'accountable',
 'acp',
 'adenosine',
 'adenoviral',
 'administrative',
 'admission',
 'adrenergic',
 'adrenoceptor',
 'aerobic',
 'aim',
 'alleviate',
 'ambulatory',
 'amyloidosis',
 'amyloidotic',
 'anaerobic',
 'anemia',
 'angiogenesis',
 'ankle',
 'anti',
 'antiarrhythmic',
 'antioxidant',
 'antiplatelet',
 'aortic',
 'apoptosis',
 'armamentarium',
 'arrhythmia',
 'arterial',
 'arthritis',
 'atherosclerosis',
 'atp',
 'atrial',
 'atrioventricular',
 'axis',
 'balance',
 'baroreflex',
 'bat',
 'bax',
 'be',
 'beat',
 'beta',
 'bleed',
 'blocker',
 'blood',
 'bmi',
 'bottleneck',
 'bpm',
 'brachial',
 'breast',
 'breath',
 'breathing',
 'breathlessness',
 'bridging',
 'bromodomain',
 'budget',
 'calreticulin',
 'cancer',
 'carbohydrate',
 'cardiac',
 'cardiography',
 'cardiologist',
 'cardiology',
 'cardiomyiopathy',
 'cardiomyocyte',
 'cardiomyocytes',
 'cardiomyopathic',
 'cardiomyopathy',
 'cardioprotection',
 'cardiopulmonary',
 'cardiorespiratory',
 'ca

In [29]:
set(gts_lem)

{'mis',
 'mitral regurgitation',
 'lv end diastolic dimension',
 'ref',
 'neurohormonal stimulation',
 'multivariate adjust odd ratio',
 'prognostic',
 'tunel',
 'pkc βii',
 'western blot',
 'mr proanp',
 'anti inflammatory',
 'hospital',
 'chamber patterning',
 'biventricular dysfunction',
 'emergency room',
 'poc',
 'arterial pressure',
 'leave ventricular base',
 'xanthine oxidase',
 'intensive care unit',
 'metoprolol succinate',
 'respiratory gas analysis',
 'histone deacetylase',
 'hypertrophic',
 'carvedilol',
 'lv assist device implantation',
 'lcz696',
 'tachypacing induce hf',
 'inoperable',
 'wilcoxon match pair sign rank test',
 'arm cuff',
 'leave anterior descend artery',
 'leave bundle branch block pattern',
 'nyha iii',
 'antiadrenergic',
 'neoangiogenesis',
 'lvas',
 'medical history',
 'cpg',
 'leave ventricular diastolic function',
 'serum potassium',
 'nuclear factor of activate t cell',
 'concentric remodeling',
 'molecular pathway',
 'mild hf',
 'heart transplanta

In [30]:
count_freq(set(preds_lem), set(gts_lem))

Counter({1: 430, 2: 1, 3: 1})
Counter({1: 857, 2: 672, 3: 379, 4: 148, 5: 46, 6: 30, 7: 13, 8: 3, 10: 1})


NLTK

In [15]:
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# port = PorterStemmer()
# wnl = WordNetLemmatizer()

# preds_stem = [port.stem(i) for i in preds]
# gts_stem = [port.stem(i) for i in gts]

# print('After stemming')
# print(len(set(preds_stem)))
# print(len(set(gts_stem)))
# print(evaluation_metrics(list(set(preds_stem)), list(set(gts_stem))))

# preds_lem = [wnl.lemmatize(i) for i in preds]
# gts_lem = [wnl.lemmatize(i) for i in gts]

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

# preds_com = [port.stem(wnl.lemmatize(i)) for i in preds]
# gts_com = [port.stem(wnl.lemmatize(i)) for i in gts]

# print('After combined')
# print(len(set(preds_com)))
# print(len(set((gts_com))))
# evaluation_metrics(list(set(preds_com)), list(set(gts_com)))

# preds_ =[x for x in preds if len(x) >= 2]
# gts_ =[x for x in gts if len(x) >= 2]
# print(len(set(preds_)))
# print(len(gts))
# evaluation_metrics(set(preds_), gts)

In [ ]:
en_gt

In [ ]:
en_gt['len']  = [len(x) for x in en_gt[0]]
en_gt[en_gt['len']>1][[0]]